In [1]:
import logging
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
#构建corpus
import os

path = 'D:/毕业设计/code/data/'  # 待处理的数据

fileList = os.listdir(path)
docs = [] #文档集合
for file in fileList:
    filePath = os.path.join(path, file)
    f = open(filePath, encoding='utf-8')
    content = f.read()
    f.close()
    docs.append(content)
print(len(docs))
print(docs[0][:500])

8
The move allegedly happened after Apple CEO Tim Cook criticized the social media company. Facebook CEO Mark Zuckerberg. It looks like Mark Zuckerberg can hold a grudge. The Facebook CEO reportedly ordered his management team to start using only Android phones after Apple CEO Tim Cook made public comments about the social media company's Cambridge Analytica data scandal, according to a report Wednesday in The New York Times. "We're not going to traffic in your personal life," Cook said, referenci


In [3]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]


D:\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]


In [5]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)


2022-04-21 15:46:24,663 : INFO : collecting all words and their counts
2022-04-21 15:46:24,664 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2022-04-21 15:46:24,673 : INFO : collected 6047 word types from a corpus of 6365 words (unigram + bigrams) and 8 sentences
2022-04-21 15:46:24,674 : INFO : using 6047 counts as vocab in Phrases<0 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>


In [6]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=2, no_above=0.8)
print(dictionary)


2022-04-21 15:46:24,714 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-04-21 15:46:24,721 : INFO : built Dictionary(1644 unique tokens: ['a', 'about', 'according', 'account', 'added']...) from 8 documents (total 6461 corpus positions)
2022-04-21 15:46:24,723 : INFO : discarding 1237 tokens: [('a', 7), ('analytica', 1), ('and', 7), ('black', 1), ('blog', 1), ('by', 7), ('cambridge', 1), ('came', 1), ('campaign', 1), ('chose', 1)]...
2022-04-21 15:46:24,724 : INFO : keeping 407 tokens which were in no less than 2 and no more than 6 (=80.0%) documents
2022-04-21 15:46:24,725 : INFO : resulting dictionary: Dictionary(407 unique tokens: ['about', 'according', 'account', 'added', 'after']...)


Dictionary(407 unique tokens: ['about', 'according', 'account', 'added', 'after']...)


In [7]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [8]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 407
Number of documents: 8


In [9]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 4
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-04-21 15:46:24,779 : INFO : using autotuned alpha, starting with [0.25, 0.25, 0.25, 0.25]
2022-04-21 15:46:24,781 : INFO : using serial LDA version on this node
2022-04-21 15:46:24,782 : INFO : running online (multi-pass) LDA training, 4 topics, 20 passes over the supplied corpus of 8 documents, updating model once every 8 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-04-21 15:46:24,782 : INFO : PROGRESS: pass 0, at document #8/8
2022-04-21 15:46:24,838 : WARNING : updated prior is not positive
2022-04-21 15:46:24,839 : INFO : optimized alpha [0.25, 0.25, 0.25, 0.25]
2022-04-21 15:46:24,840 : INFO : topic #0 (0.250): 0.032*"stock" + 0.019*"technology" + 0.016*"on" + 0.015*"ltd" + 0.014*"it" + 0.014*"at" + 0.013*"share" + 0.012*"earnings" + 0.012*"inc" + 0.011*"that"
2022-04-21 15:46:24,841 : INFO : topic #1 (0.250): 0.034*"stock" + 0.028*"price" + 0.025*"ltd" + 0.025*"holding" + 0.022*"were" + 0.019*"share" + 0.018

2022-04-21 15:46:24,912 : INFO : topic #1 (0.077): 0.041*"stock" + 0.030*"price" + 0.029*"ltd" + 0.026*"holding" + 0.022*"were" + 0.020*"share" + 0.019*"technology" + 0.019*"at" + 0.018*"inc" + 0.018*"purchase"
2022-04-21 15:46:24,913 : INFO : topic #2 (0.115): 0.025*"it" + 0.022*"apple" + 0.017*"we" + 0.015*"that" + 0.014*"facebook" + 0.014*"on" + 0.013*"with" + 0.011*"an" + 0.011*"from" + 0.011*"ha"
2022-04-21 15:46:24,914 : INFO : topic #3 (0.016): 0.002*"one" + 0.002*"uk" + 0.002*"london" + 0.002*"canada" + 0.002*"number" + 0.002*"it" + 0.002*"at" + 0.002*"that" + 0.002*"with" + 0.002*"price"
2022-04-21 15:46:24,915 : INFO : topic diff=0.047841, rho=0.333333
2022-04-21 15:46:24,915 : INFO : PROGRESS: pass 8, at document #8/8
2022-04-21 15:46:24,921 : INFO : optimized alpha [0.031190353, 0.06950701, 0.10779929, 0.0149525795]
2022-04-21 15:46:24,921 : INFO : topic #0 (0.031): 0.028*"million" + 0.026*"quarter" + 0.020*"report" + 0.018*"revenue" + 0.016*"stock" + 0.015*"share" + 0.015*

2022-04-21 15:46:24,996 : INFO : topic #0 (0.022): 0.026*"million" + 0.022*"quarter" + 0.018*"report" + 0.017*"with" + 0.017*"it" + 0.016*"revenue" + 0.014*"stock" + 0.014*"share" + 0.013*"an" + 0.013*"that"
2022-04-21 15:46:24,996 : INFO : topic #1 (0.046): 0.041*"stock" + 0.030*"price" + 0.030*"ltd" + 0.025*"holding" + 0.022*"were" + 0.020*"share" + 0.020*"technology" + 0.019*"at" + 0.018*"inc" + 0.018*"purchase"
2022-04-21 15:46:24,997 : INFO : topic #2 (0.067): 0.024*"it" + 0.024*"apple" + 0.016*"we" + 0.015*"facebook" + 0.015*"that" + 0.014*"on" + 0.012*"with" + 0.010*"an" + 0.010*"from" + 0.010*"ha"
2022-04-21 15:46:24,998 : INFO : topic #3 (0.011): 0.002*"one" + 0.002*"uk" + 0.002*"london" + 0.002*"canada" + 0.002*"number" + 0.002*"it" + 0.002*"at" + 0.002*"that" + 0.002*"with" + 0.002*"an"
2022-04-21 15:46:24,999 : INFO : topic diff=0.006035, rho=0.242536
2022-04-21 15:46:25,000 : INFO : PROGRESS: pass 16, at document #8/8
2022-04-21 15:46:25,005 : INFO : optimized alpha [0.021

In [10]:
from pprint import pprint
pprint(lda_model.print_topics()) # 打印最终分类的主题

2022-04-21 15:46:25,044 : INFO : topic #0 (0.019): 0.026*"million" + 0.022*"quarter" + 0.018*"report" + 0.017*"with" + 0.017*"it" + 0.016*"revenue" + 0.013*"stock" + 0.013*"share" + 0.013*"an" + 0.013*"that"
2022-04-21 15:46:25,045 : INFO : topic #1 (0.039): 0.041*"stock" + 0.030*"price" + 0.030*"ltd" + 0.025*"holding" + 0.022*"were" + 0.020*"share" + 0.020*"technology" + 0.019*"at" + 0.018*"inc" + 0.018*"purchase"
2022-04-21 15:46:25,046 : INFO : topic #2 (0.055): 0.024*"it" + 0.024*"apple" + 0.015*"we" + 0.015*"facebook" + 0.015*"that" + 0.014*"on" + 0.012*"with" + 0.010*"an" + 0.010*"from" + 0.010*"ha"
2022-04-21 15:46:25,047 : INFO : topic #3 (0.010): 0.002*"one" + 0.002*"uk" + 0.002*"london" + 0.002*"canada" + 0.002*"number" + 0.002*"it" + 0.002*"at" + 0.002*"that" + 0.002*"with" + 0.002*"price"


[(0,
  '0.026*"million" + 0.022*"quarter" + 0.018*"report" + 0.017*"with" + '
  '0.017*"it" + 0.016*"revenue" + 0.013*"stock" + 0.013*"share" + 0.013*"an" + '
  '0.013*"that"'),
 (1,
  '0.041*"stock" + 0.030*"price" + 0.030*"ltd" + 0.025*"holding" + '
  '0.022*"were" + 0.020*"share" + 0.020*"technology" + 0.019*"at" + '
  '0.018*"inc" + 0.018*"purchase"'),
 (2,
  '0.024*"it" + 0.024*"apple" + 0.015*"we" + 0.015*"facebook" + 0.015*"that" + '
  '0.014*"on" + 0.012*"with" + 0.010*"an" + 0.010*"from" + 0.010*"ha"'),
 (3,
  '0.002*"one" + 0.002*"uk" + 0.002*"london" + 0.002*"canada" + 0.002*"number" '
  '+ 0.002*"it" + 0.002*"at" + 0.002*"that" + 0.002*"with" + 0.002*"price"')]


In [11]:
from gensim.models import CoherenceModel

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better. 

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)   # 越高越好


2022-04-21 15:46:25,068 : INFO : -5.588 per-word bound, 48.1 perplexity estimate based on a held-out corpus of 8 documents with 2942 words
2022-04-21 15:46:25,069 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate probabilities from sliding windows



Perplexity:  -5.587618053189899


2022-04-21 15:46:32,023 : INFO : 1 batches submitted to accumulate stats from 64 documents (5589 virtual)
2022-04-21 15:46:32,235 : INFO : 7 accumulators retrieved from output queue
2022-04-21 15:46:32,267 : INFO : accumulated word occurrence stats for 5686 virtual documents



Coherence Score:  0.37860950623689915


In [12]:
import pyLDAvis
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis, 'lda_genim.html')

2022-04-21 15:46:32,518 : INFO : NumExpr defaulting to 8 threads.


AttributeError: module 'pyLDAvis' has no attribute 'gensim_models'